In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Feb  5 08:50:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    29W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

##  data set


### Setup and Load dataset

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget  -b  https://files.grouplens.org/datasets/movielens/ml-25m.zip

Continuing in background, pid 1181.
Output will be written to ‘wget-log.1’.


In [ ]:
!unzip -o "ml-25m.zip"  -d  "/content"

Archive:  ml-25m.zip
  inflating: /content/ml-25m/tags.csv  
  inflating: /content/ml-25m/links.csv  
  inflating: /content/ml-25m/README.txt  
  inflating: /content/ml-25m/ratings.csv  
  inflating: /content/ml-25m/genome-tags.csv  
  inflating: /content/ml-25m/genome-scores.csv  
  inflating: /content/ml-25m/movies.csv  


In [ ]:
data = pd.read_csv('ml-25m/ratings.csv')#, sep='::', header=0, skipinitialspace=True)
data.dropna(inplace=True)


In [ ]:
data.columns =[ 'user', 'item', 'label','timestamp']

##  LibRecommender

In [ ]:
!pip install numpy==1.22

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [ ]:
!pip install LibRecommender==0.12.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20443 sha256=b962a05c63714cafc276d2c4f3410dd3f7eb73b401c1b40d1dfb9c23e163341b
  Stored in directory: /root/.cache/pip/wheels/6a/65/c4/d27eeee9ba3fc150a0dae150519591103b9e0dbffde3ae77dc
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3530 sha256=488e4549217b9b46294b5cbc21522a6419cdc1c7c0afb7245b00efc8e7c5bd6c
  Stored in directory: /root/.cache/pip/wheels/ba/d9/a0/ddd93af16d5855dd9bad417623e70948fdac119d1d34fb17c8
Successfully built fst-pso miniful
  Attempting uninstall: gen

In [ ]:

import tensorflow as tf

from libreco.data import DatasetFeat, DataInfo
from libreco.data import split_by_ratio_chrono,split_by_ratio
from libreco.algorithms import DeepFM,YouTubeRanking
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

col_user = ["user"]
col_item = ["item"]
col_dense = ["label"]

In [ ]:
train, test = split_by_ratio(data, test_size=0.2)

In [ ]:
train_data, data_info = DatasetFeat.build_trainset(
        train,
       
        shuffle=False,
    )

In [ ]:
test_data = DatasetFeat.build_testset(test, shuffle=False)

In [ ]:
eval_data = DatasetFeat.build_testset(test)
print(data_info)


n_users: 162541, n_items: 37970, data sparsity: 0.3241 %


In [ ]:
import torch
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.algorithms import (
   ALS
)


    # # only do negative sampling on eval data
    # eval_data.build_negative_samples(
    #     data_info, item_gen_mode="random", num_neg=1, seed=2222
    # )
metrics = [  
        "precision",
        "recall",
        "map",
        "ndcg",
    ]


## als working with 25M

In [ ]:
train_data.build_negative_samples(
        data_info, item_gen_mode="random", num_neg=1, seed=2020
    )
eval_data.build_negative_samples(
        data_info, item_gen_mode="random", num_neg=1, seed=2222
    )

random neg item sampling elapsed: 20.864s
random neg item sampling elapsed: 5.937s


In [ ]:
als = ALS(
        "ranking",
        data_info,
        embed_size=16,
        n_epochs=2,
        reg=5.0,
        alpha=10,
        use_cg=False,
        n_threads=1,
        seed=42,
    )
als.fit(
        train_data,
        verbose=2,
        eval_data=eval_data,
        metrics=metrics,
    )

Training start time: 2023-02-05 08:58:13
Epoch 1 elapsed: 15.005s


eval_rec: 100%|██████████| 162519/162519 [04:23<00:00, 616.98it/s]


	 eval precision@10: 0.0279
	 eval recall@10: 0.0190
	 eval map@10: 0.0604
	 eval ndcg@10: 0.0884
Epoch 2 elapsed: 14.657s


eval_rec: 100%|██████████| 162519/162519 [04:07<00:00, 655.48it/s]


	 eval precision@10: 0.0127
	 eval recall@10: 0.0141
	 eval map@10: 0.0241
	 eval ndcg@10: 0.0382
